# Append new data to SUMup datasets

#### Created 26 May 2022 by Megan Thompson-Munson

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib_inline.backend_inline as mplbi
mplbi.set_matplotlib_formats('svg')

In [124]:
path = '/Users/metm9666/Dropbox/research/proj_SUMup/2022_release/'

## Subdataset: Density

### Open existing, fixed SUMup dataset

In [125]:
density = pd.read_csv(path+'data/density/SUMup_density_2021-fixed.csv')
density

,Profile,Citation,Method,Date,Timestamp,Latitude,Longitude,Elevation,SDOS_Flag,Start_Depth,Stop_Depth,Midpoint,Density,Error
0,1,1,4,20020000,2002-01-01,-89.93330,144.393799,2808.0,0,-9999.0,-9999.0,3.305,420.97000,-9999.0
1,1,1,4,20020000,2002-01-01,-89.93330,144.393799,2808.0,0,-9999.0,-9999.0,4.255,414.75998,-9999.0
2,1,1,4,20020000,2002-01-01,-89.93330,144.393799,2808.0,0,-9999.0,-9999.0,5.245,418.84000,-9999.0
3,1,1,4,20020000,2002-01-01,-89.93330,144.393799,2808.0,0,-9999.0,-9999.0,6.220,437.48000,-9999.0
4,1,1,4,20020000,2002-01-01,-89.93330,144.393799,2808.0,0,-9999.0,-9999.0,7.205,445.80002,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2282664,1944,193,1,20181210,2018-12-10,-70.80362,-71.714775,-9999.0,0,0.6,0.7,0.650,408.00000,-9999.0
2282665,1944,193,1,20181210,2018-12-10,-70.80362,-71.714775,-9999.0,0,0.7,0.8,0.750,344.00000,-9999.0
2282666,1944,193,1,20181210,2018-12-10,-70.80362,-71.714775,-9999.0,0,0.8,0.9,0.850,385.00000,-9999.0
2282667,1944,193,1,20181210,2018-12-10,-70.80362,-71.714775,-9999.0,0,0.9,1.0,0.950,370.00000,-9999.0


### Add Rennermalm et al. (2021)

https://arcticdata.io/catalog/view/doi%3A10.18739%2FA2Q52FD98

In [126]:
# Read in meta data
metadata = pd.read_csv(path+'new_data/density-rennermalm/Core_meta_data.csv',
                       skiprows=8,nrows=24,
                       usecols=[0,1,2,3,5,6,7])

# Convert separte date columns into single timestamp column
dm = metadata['Date (first day, DD-MMM)'].values
y = metadata['Year'].values
dates = []
for i in range(len(dm)):
    daymonth = dm[i]
    year = y[i]
    date = dm[i]+'-'+str(y[i])
    dates.append(pd.to_datetime(date).date())
metadata['Date'] = dates

In [127]:
# Get list of site names
sites = metadata['Site name']
core_numbers = metadata['Core #']

# Set column order
cols = ['Profile','Citation','Method','Date','Timestamp','Latitude','Longitude','Elevation','SDOS_Flag',
        'Start_Depth','Stop_Depth','Midpoint','Density','Error']

# Loop through and get each site file
dfs = []
for i in range(len(sites)):
    site = sites[i].replace(' ','').replace('-','').replace('_','')
    year = str(y[i])
    core_number = str(core_numbers[i])
    if core_number == 'nan':
        fname = site+'_'+year+'.csv'
    else:
        fname = site+'_'+year+'_'+core_number[0]+'.csv'
    
    # Open dataframe with relevant columns
    df = pd.read_csv(path+'new_data/density-rennermalm/'+fname,
                     skiprows=6,usecols=[0,1,13],names=['Start_Depth','Stop_Depth','Density'])
    
    # Add metadata to dataframe
    df['Latitude'] = metadata['Latitude (N)'][i]
    df['Longitude'] = metadata['Longitude (E)'][i]
    df['Elevation'] = metadata['Elevation (m a.s.l)'][i]
    df['Timestamp'] = metadata['Date'][i]
    df['Date'] = str(metadata['Date'][i])[0:10].replace('-','')
    df['Midpoint'] = -9999
    df['SDOS_Flag'] = 0
    df['Error'] = -9999
    df['Method'] = 4
    df['Citation'] = density.Citation.values[-1]+1
    df['Profile'] = density.Profile.values[-1]+i+1
    
    # Rearrange columns
    df = df[cols]
    dfs.append(df)

rennermalm = pd.concat(dfs).reset_index(drop=True)

In [128]:
# Print statistics
print('N measurements: {}'.format(len(rennermalm)))
print('N profiles: {}'.format(len(rennermalm.Profile.unique())))

N measurements: 2674
N profiles: 24


In [129]:
# Add to dataset
density = pd.concat([density,rennermalm]).reset_index(drop=True)

### Add Niwano et al. 2020

https://ads.nipr.ac.jp/data/meta/A20200911-001

In [130]:
# Read in data
xls = pd.ExcelFile(path+'new_data/density-sumup-data-SIGMA-Traverse2018_density.xlsx')
df = pd.read_excel(xls,'Density on Ice Sheets',
                   skiprows=[0,1,3],usecols=[0,1,2,3,4,5,6,7,8,9,10])

# Convert dates to pandas timestamp
df['Timestamp'] = [d.date() for d in df.Date]

# Loop through and change dates to string, add profile, and add citation
dfis = []
for i in range(len(df.ID.unique())):
    dfi = df[df.ID==df.ID.unique()[i]].reset_index(drop=True)
    dfi['Date'] = dfi['Date'].dt.strftime('%Y%m%d')
    dfi['SDOS_Flag'] = 0
    dfi['Citation'] = density.Citation.values[-1]+1
    dfi['Profile'] = density.Profile.values[-1]+i+1
    dfi['Start_Depth'] = dfi['Start Depth']
    dfi['Stop_Depth'] = dfi['Stop Depth']
    dfi = dfi[cols]
    dfis.append(dfi)
    
niwano = pd.concat(dfis).reset_index(drop=True)

In [131]:
# Print statistics
print('N measurements: {}'.format(len(niwano)))
print('N profiles: {}'.format(len(niwano.Profile.unique())))

N measurements: 104
N profiles: 4


In [132]:
# Add to dataset
density = pd.concat([density,niwano]).reset_index(drop=True)

### Add Weinhart et al. 2021
https://doi.org/10.1594/PANGAEA.928079

In [133]:
# Open dataframe
df = pd.read_table(path+'new_data/density-EOP_snow_density.txt',skiprows=44,sep='\t')
df = df.sort_values('Location').reset_index()

# These are 1-m density measurements, so add the midpoint as 0.5 m; and add other fields
df['Midpoint'] = 0.5
df['SDOS_Flag'] = 0
df['Start_Depth'] = -9999
df['Stop_Depth'] = -9999
df['Citation'] = density.Citation.values[-1]+1
df['Profile'] = density.Profile.values[-1]+df.Location
df['Elevation'] = df['Elevation [m a.s.l.]']
df['Timestamp'] = pd.to_datetime(df['Date/Time'])
df['Date'] = df['Timestamp'].dt.strftime('%Y%m%d')
df['Timestamp'] = [d.date() for d in df.Timestamp]
df['Density'] = df['Density snow mean [kg/m**3]']
df['Error'] = df['Density snow std dev [±]']
df['Method'] = 10

weinhart = df[cols]

In [134]:
# Print statistics
print('N measurements: {}'.format(len(weinhart)))
print('N profiles: {}'.format(len(weinhart.Profile.unique())))

N measurements: 22
N profiles: 22


In [135]:
# Add to dataset
density = pd.concat([density,weinhart]).reset_index(drop=True)

### Add Wever et al. (2022)

Wever, Nander; Keenan, Eric; Kausch, Thore; Lehning, Michael (2022). SnowMicroPen measurements and manual snowpits from Dronning Maud Land, East Antarctica. EnviDat. doi:10.16904/envidat.331

In [136]:
# Read in data
xls = pd.ExcelFile(path+'new_data/data-Nander.xlsx')
df = pd.read_excel(xls,'Density on Ice Sheets',
                   skiprows=[0,1,3],usecols=[0,1,2,3,4,5,6,7,8,9,10])

# Convert dates to pandas timestamp
df['Timestamp'] = [d.date() for d in df.Date]

# Loop through and change dates to string, add profile, and add citation
dfis = []
for i in range(len(df.ID.unique())):
    dfi = df[df.ID==df.ID.unique()[i]].reset_index(drop=True)
    dfi['Date'] = dfi['Date'].dt.strftime('%Y%m%d')
    dfi['SDOS_Flag'] = 0
    dfi['Citation'] = density.Citation.values[-1]+1
    dfi['Profile'] = density.Profile.values[-1]+i+1
    dfi['Start_Depth'] = dfi['Start Depth']
    dfi['Stop_Depth'] = dfi['Stop Depth']
    dfi = dfi[cols]
    dfis.append(dfi)
    
wever = pd.concat(dfis).reset_index(drop=True)

In [137]:
# Print statistics
print('N measurements: {}'.format(len(wever)))
print('N profiles: {}'.format(len(wever.Profile.unique())))

N measurements: 119
N profiles: 5


In [138]:
# Add to dataset
density = pd.concat([density,wever]).reset_index(drop=True)

### Save new dataset

In [140]:
density.to_csv(path+'data/density/SUMup_density_2022.csv',index=False)